## 任务：京东用户购买意向预测

### 故事背景：
京东作为中国最大的自营式电商，在保持高速发展的同时，沉淀了数亿的忠实用户，积累了海量的真实数据。如何从历史数据中找出规律，去预测用户未来的购买需求，让最合适的商品遇见最需要的人，是大数据应用在精准营销中的关键问题，也是所有电商平台在做智能化升级时所需要的核心技术。

以京东商城真实的用户、商品和行为数据（脱敏后）为基础，通过数据挖掘的技术和机器学习的算法，构建用户购买商品的预测模型，输出高潜用户和目标商品的匹配结果，为精准营销提供高质量的目标群体。

目标：使用京东多个品类下商品的历史销售数据，构建算法模型，预测用户在未来5天内，对某个目标品类下商品的购买意向。

### 数据集：
这里涉及到的数据集是京东的数据集:

* JData_User.csv 用户数据集 105,321个用户
* JData_Comment.csv 商品评论 558,552条记录
* JData_Product.csv 预测商品集合 24,187条记录
* JData_Action_201602.csv 2月份行为交互记录 11,485,424条记录
* JData_Action_201603.csv 3月份行为交互记录 25,916,378条记录
* JData_Action_201604.csv 4月份行为交互记录 13,199,934条记录

**JData_User.csv用户数据**

|字段|意义|备注|
|-|-|-|
|user_id|用户id|脱敏|
|age|年龄|-1表未知|
|sex|性别|0男，1女，2未知|
|user_lv_cd|用户等级|级别枚举，越高级别越大|
|user_reg_tm|用户注册日期|粒度到天|

**JData_Comment.csv评论数据**

|字段|意义|备注|
|-|-|-|
|dt|截止时间|天，到2016-02-01|
|sku_id|商品编号|脱敏|
|comment_num|累积评论数分段|0表示无评论，1表是1条，2表示2-10条，3表示11-50条，5表示大于50条|
|has_bad_comment|是否有差评|0表示无，1表示有|
|bad_comment_rate|差评率|差评数占总评论数的比率|

**JData_Product.csv商品数据**

|字段|意义|备注|
|-|-|-|
|sku_id|商品编号|脱敏|
|a1|属性1|枚举，-1表未知|
|a2|属性2|枚举，-1表未知|
|a3|属性3|枚举，-1表未知|
|cate|品牌ID|脱敏|
|brand|品牌ID|脱敏|

**JData_Action_xx.csv商品数据**

|字段|意义|备注|
|-|-|-|
|user_id|用户ID|脱敏|
|sku_id|商品编号|脱敏|
|time|行为时间||
|model_id|点击板块的编号|脱敏|
|type|行为类型|1.浏览商品详情页;2.加入购物车;3.购物车删除;4.下单;5.关注;6.点击;|
|cate|品牌ID|脱敏|
|brand|品牌ID|脱敏|

### 数据挖掘流程：
（一）.数据清洗
1. 数据集完整性验证
2. 数据集中是否存在缺失值
3. 数据集中各特征数值应该如何处理
4. 哪些数据是我们想要的，哪些是可以过滤掉的
5. 将有价值数据信息做成新的数据源
6. 去除无行为交互的商品和用户
7. 去掉浏览量很大而购买量很少的用户(惰性用户或爬虫用户)

（二）.数据理解与分析
1. 掌握各个特征的含义
2. 观察数据有哪些特点，是否可利用来建模
3. 可视化展示便于分析
4. 用户的购买意向是否随着时间等因素变化
（三）.特征提取
1. 基于清洗后的数据集哪些特征是有价值
2. 分别对用户与商品以及其之间构成的行为进行特征提取
3. 行为因素中哪些是核心？如何提取？
4. 瞬时行为特征or累计行为特征？

（四）.模型建立
1. 使用机器学习算法进行预测
2. 参数设置与调节
3. 数据集切分

### 数据集完整性验证
首先检查JData_User中的用户和JData_Dction中的用户是否一致，保证行为数据中锁产生的行为均由用户数据中的用户产生。

思路：利用pd.Merge连接sku和Action中的sku，观测Action中的数据是否减少Example：

In [1]:
# 测试方法
import pandas as pd
df1 = pd.DataFrame({'sku':['a','a','e','c'], 'data':[1,1,2,3]})
df2 = pd.DataFrame({'sku':['a','b','c']})
print(pd.merge(df1,df2))

  sku  data
0   a     1
1   a     1
2   c     3


结果只会打印两者共有的部分

In [2]:
#数据集验证
def user_action_check():
    df_user = pd.read_csv('data/JData_User.csv',encoding='gbk')
    df_sku = df_user.loc[:,'user_id'].to_frame()
    df_month2 = pd.read_csv('data/JData_Action_201602.csv',encoding='gbk')
    # pd.merge(df_sku,df_month2) 会以user_id字段为基准取两个df的交集 不是取并集，这样才能证明 action中的userid 都在df_user里面
    print ('Is action of Feb. from User file? ', len(df_month2) == len(pd.merge(df_sku,df_month2))) 
    df_month3 = pd.read_csv('data/JData_Action_201603.csv',encoding='gbk')
    print ('Is action of Mar. from User file? ', len(df_month3) == len(pd.merge(df_sku,df_month3)))
    df_month4 = pd.read_csv('data/JData_Action_201604.csv',encoding='gbk')
    print ('Is action of Apr. from User file? ', len(df_month4) == len(pd.merge(df_sku,df_month4)))

user_action_check() 

Is action of Feb. from User file?  True
Is action of Mar. from User file?  True
Is action of Apr. from User file?  True


结论：User数据集中的用户和交互行为数据集中的用户完全一致

根据merge前后的数据量对，能保障Action中的用户ID是User中的ID的子集

### 检查是否有重复记录
除去各个数据文件中完全重复的记录,可能解释是重复数据是有意义的，比如用户同时购买多件商品，同时添加多个数量的商品到购物车等…

In [3]:
#重复数据
def deduplicate(filepath, filename, newpath):
    df_file = pd.read_csv(filepath,encoding='gbk')       
    before = df_file.shape[0]
    df_file.drop_duplicates(inplace=True) # 列相同认为是重复 inplace=True表示在原来的DataFrame上删除重复项4
    after = df_file.shape[0]
    n_dup = before-after  # 查看前后差值
    print ('Number of duplicate records for ' + filename + ' is: ' + str(n_dup))
    if n_dup != 0:
        df_file.to_csv(newpath, index=None)
    else:
        print ('Number duplicate records in ' + filename)

In [4]:
deduplicate('data/JData_Action_201602.csv', 'Feb. action', 'data/JData_Action_201602_dedup.csv')
deduplicate('data/JData_Action_201603.csv', 'Mar. action', 'data/JData_Action_201603_dedup.csv')
deduplicate('data/JData_Action_201604.csv', 'Feb. action', 'data/JData_Action_201604_dedup.csv')
deduplicate('data/JData_Comment.csv', 'Comment', 'data/JData_Comment_dedup.csv')
deduplicate('data/JData_Product.csv', 'Product', 'data/JData_Product_dedup.csv')
deduplicate('data/JData_User.csv', 'User', 'data/JData_User_dedup.csv')

Number of duplicate records for Feb. action is: 2756093
Number of duplicate records for Mar. action is: 7085038
Number of duplicate records for Feb. action is: 3672710
Number of duplicate records for Comment is: 0
Number duplicate records in Comment
Number of duplicate records for Product is: 0
Number duplicate records in Product
Number of duplicate records for User is: 0
Number duplicate records in User


In [6]:
# 查看重复数据
df_month2 = pd.read_csv('data/JData_Action_201602.csv',encoding='gbk')
IsDuplicated = df_month2.duplicated()
df_d = df_month2[IsDuplicated]
df_d.groupby('type').count()  # 发现重复数据大多数都是由于浏览（1），或者点击(6)产生

,user_id,sku_id,time,model_id,cate,brand
type,,,,,,
1,2176378,2176378,2176378,0,2176378,2176378
2,636,636,636,0,636,636
3,1464,1464,1464,0,1464,1464
4,37,37,37,0,37,37
5,1981,1981,1981,0,1981,1981
6,575597,575597,575597,545054,575597,575597
